### Regex main building blocks:
* Literal Characters: The most basic regular expression consists of a single literal character.
* Special Characters: We have 11 special characters.
    1. backslash [\\]: 
    2. caret [^]: matches the position before the first character in the string
    3. dollar sign [$]: matches right after the last character in the string
    4. dot [.]: The dot matches a single character, without caring what that character is. The only exception are line break characters
    5. pipe symbol [|]: alternation to match a single regular expression out of several possible regular expressions
    6. question mark [?]: makes the preceding token in the regular expression optional, match zero times or once
    7. star [*]: match the preceding token zero or more times
    8. plus sign [+]: match the preceding token once or more
    9. parenthesis [()]: you can group a part of the regular expression together
    10. square bracket [\[\]]: for character clases, match only one out of several characters ex. [a-z]
    11. curly brace [{}]: allows you to specify how many times a token can be repeated. The syntax is {min,max}
* Shorthand Character Classes: 
    1. \d : [0-9]
    2. \w: [A-Za-z0-9_]
    3. \s: [ \t\r\n\f]
    4. dot [.]:
    5. pipe symbol [|]:
    6. question mark [?]:


In [26]:
# How to represent a NFA maybe as
# 2d array representing transitions next_state = transition_table[row][col];
# events are rows & states are columns
# how to match abc, this is how the transition table should look like
# last state is always accept state?? maybe
#   s0 s1 s2 s3
# a 1  N  N
# b N  2  N
# c N  N  3
class State:
    def __init__(self, size, name):
        self.transitions = [None] * size  # Initialize the transitions list with None representing a failure state
        self.name = name
        
    def setTransitions(self, range: range, nextStateIdx: int) -> None:
        for i in range:
            self.transitions[i] = nextStateIdx 
        
class EngineNFA:
    def __init__(self, alphabet):
        self.transitions_table = [] 
        self.alphabet = alphabet # the language this NFA can describe
        
    def add_state(self, name, transition: range, nextStateIdx: int) -> None:
        new_state = State(len(self.alphabet.keys()), name)  # Create a new instance of State
        new_state.setTransitions(transition, nextStateIdx)
        self.transitions_table.append(new_state)  # Add it to 
        
    def get_state(self, index):
        if 0 <= index < len(self.transitions_table):
            return self.transitions_table[index]  # Return the State object at the given index
        return None  # Return None if index is out of range
    
    def dump(self):
        for idx, symbol in enumerate(self.alphabet.keys()):
            print(f"{self.alphabet[symbol]} => ", end=" ")
            for state in self.transitions_table:
                print(state.transitions[idx], end=" ")
            print('\n')
    
def main():
    alphabet = {i: chr(i) for i in range(ord('a'), ord('c') + 1)}
    NFA = EngineNFA(alphabet)
    # state s0
    NFA.add_state("s1",  range(0), None)
    
    # state s1
    NFA.add_state("s1", range(0,1), 2)
    
    # state s0
    NFA.add_state("s2", range(1,2), 3)
    
    # state s3
    NFA.add_state("s3", range(2,3), 4)
    
    NFA.dump()
main()


a =>  None 2 None None 

b =>  None None 3 None 

c =>  None None None 4 

